Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# Logistic regression with regularization and SGD
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    regularizer = beta * tf.reduce_mean(tf.nn.l2_loss(weights))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regularizer)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.837475
Minibatch accuracy: 13.3%
Validation accuracy: 15.1%
Test accuracy: 15.4%
Minibatch loss at step 500: 1.112743
Minibatch accuracy: 72.7%
Validation accuracy: 79.5%
Test accuracy: 86.8%
Minibatch loss at step 1000: 0.797030
Minibatch accuracy: 82.8%
Validation accuracy: 80.6%
Test accuracy: 87.7%
Minibatch loss at step 1500: 0.621444
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Test accuracy: 87.7%
Minibatch loss at step 2000: 0.683667
Minibatch accuracy: 84.4%
Validation accuracy: 81.6%
Test accuracy: 89.0%
Minibatch loss at step 2500: 0.825852
Minibatch accuracy: 79.7%
Validation accuracy: 81.9%
Test accuracy: 89.1%
Minibatch loss at step 3000: 0.743522
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Test accuracy: 88.9%


In [7]:
# 1-layer NN with regularization and SGD
batch_size = 128
beta = 0.01
num_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
    logits2 = tf.matmul(logits1, weights2) + biases2
    
    regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2) + beta * regularizer)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits2)
    
    valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights2) + biases2)
    
    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights2) + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3517.136230
Minibatch accuracy: 2.3%
Validation accuracy: 27.6%
Test accuracy: 29.8%
Minibatch loss at step 500: 21.421618
Minibatch accuracy: 77.3%
Validation accuracy: 84.2%
Test accuracy: 90.7%
Minibatch loss at step 1000: 0.918854
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Test accuracy: 89.8%
Minibatch loss at step 1500: 0.614263
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Test accuracy: 90.0%
Minibatch loss at step 2000: 0.667985
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Test accuracy: 90.7%
Minibatch loss at step 2500: 0.718770
Minibatch accuracy: 83.6%
Validation accuracy: 84.3%
Test accuracy: 90.9%
Minibatch loss at step 3000: 0.721170
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Test accuracy: 90.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
# overfitting
num_steps = 3001

train_dataset_red = train_dataset[:500,:]
train_labels_red = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_red.shape[0] - batch_size)
        batch_data = train_dataset_red[offset:(offset+batch_size),:]
        batch_labels = train_labels_red[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3525.413086
Minibatch accuracy: 14.8%
Validation accuracy: 32.9%
Test accuracy: 35.3%
Minibatch loss at step 500: 21.198454
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Test accuracy: 83.5%
Minibatch loss at step 1000: 0.477595
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Test accuracy: 85.3%
Minibatch loss at step 1500: 0.298238
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Test accuracy: 85.4%
Minibatch loss at step 2000: 0.290195
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Test accuracy: 85.4%
Minibatch loss at step 2500: 0.280412
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Test accuracy: 85.5%
Minibatch loss at step 3000: 0.271898
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Test accuracy: 85.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [6]:
# 1-layer NN with dropout and SGD
batch_size = 128
beta = 0.01
num_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    keep_prob = tf.placeholder("float")
    logits1 = tf.nn.dropout(logits1, keep_prob)
    
    logits2 = tf.matmul(logits1, weights2) + biases2
    
    regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
        
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2))
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits2)
    
    valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights2) + biases2)
    
    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights2) + biases2)

In [11]:
# dropout
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.8}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3488.135254
Minibatch accuracy: 16.4%
Validation accuracy: 34.0%
Test accuracy: 37.6%
Minibatch loss at step 500: 21.505989
Minibatch accuracy: 75.8%
Validation accuracy: 84.7%
Test accuracy: 90.8%
Minibatch loss at step 1000: 0.942976
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Test accuracy: 90.0%
Minibatch loss at step 1500: 0.638839
Minibatch accuracy: 84.4%
Validation accuracy: 83.1%
Test accuracy: 89.7%
Minibatch loss at step 2000: 0.676060
Minibatch accuracy: 88.3%
Validation accuracy: 84.0%
Test accuracy: 90.5%
Minibatch loss at step 2500: 0.742685
Minibatch accuracy: 85.2%
Validation accuracy: 84.2%
Test accuracy: 90.8%
Minibatch loss at step 3000: 0.746780
Minibatch accuracy: 82.0%
Validation accuracy: 84.0%
Test accuracy: 90.5%


In [10]:
# overfitting with dropout
num_steps = 3001

train_dataset_red = train_dataset[:500,:]
train_labels_red = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_red.shape[0] - batch_size)
        batch_data = train_dataset_red[offset:(offset+batch_size),:]
        batch_labels = train_labels_red[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.8}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3592.949951
Minibatch accuracy: 10.2%
Validation accuracy: 23.4%
Test accuracy: 25.4%
Minibatch loss at step 500: 21.161150
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Test accuracy: 84.9%
Minibatch loss at step 1000: 0.509036
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Test accuracy: 85.2%
Minibatch loss at step 1500: 0.330097
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Test accuracy: 85.4%
Minibatch loss at step 2000: 0.326659
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 85.4%
Minibatch loss at step 2500: 0.297102
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 85.7%
Minibatch loss at step 3000: 0.292471
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 85.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
